In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_PredNTS_Classification_DLNN_Kgap_CNN"
outPath = "Results"
foldName = "folds.pickle"

epochs = 50
batch_size = 32
shuffle = True
seed = None

input_data_folder = "PredNTS_MathFeature_ENC"

monitor = 'val_accuracy'

In [2]:
kgap_max = 4

train_data_filename = 'Training-datasets-PredNTS_kgap_{}.csv'
indpe_data_filename = 'independent-dataset-PredNTS_kgap_{}.csv'

In [3]:
import os 
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score, classification_report, matthews_corrcoef

import math

In [4]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
# ##################################################################################
# ##### define all CUSTOM functions
# ##################################################################################

# def one_hot_encode_nt(sequence, char_dict):
    
#     seq_encoded = np.zeros((len(sequence),len(char_dict)))
    
#     i = 0
#     for single_character in sequence:
#         if(single_character.upper() in char_dict.keys()):
#             seq_encoded[i][char_dict[single_character.upper()]] = 1
#             i = i+1
#         else:
#             raise ValueError('Incorrect character in NT sequence: '+sequence)
#     return seq_encoded

In [6]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

In [7]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred)
    return y_pred

In [8]:
# epochs = 200
# batch_size = 16

# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_Classifier(input_vec_shape,
#                     dense_decode_units = 10, ## Dense layer parameters,
#                     dense_layers = 2,
#                     prob = 0.5, learn_rate = 0.0005, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
#     beta = 0.001
    
#     input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
#     ######################################################################################################
#     ########  CNN layers  ################################################################################
#     ######################################################################################################
    
#     cnn = tf.keras.layers.Conv1D(filters = 100, 
#                                  kernel_size = 100,
#                                  kernel_regularizer = tf.keras.regularizers.l2(beta)
#                                 )(input1)
# #     cnn = tf.keras.layers.Activation('relu')(cnn)
#     cnn = tf.keras.layers.MaxPool1D(pool_size = 10, strides = 10)(cnn)
#     cnn = tf.keras.layers.Dropout(prob)(cnn)
    
#     cnn = tf.keras.layers.Conv1D(filters = 100, 
#                                  kernel_size = 50,
#                                  kernel_regularizer = tf.keras.regularizers.l2(beta)
#                                 )(cnn)
# #     cnn = tf.keras.layers.Activation('relu')(cnn)
#     cnn = tf.keras.layers.MaxPool1D(pool_size = 5, strides = 5)(cnn)
#     cnn = tf.keras.layers.Dropout(prob)(cnn)
    
#     cnn = tf.keras.layers.Flatten()(cnn)
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta),
#                               activation = 'relu'
#                              )(cnn)
#     y = tf.keras.layers.BatchNormalization()(y)
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid'
#                              )(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss)

#     return model

In [9]:
epochs = 100
batch_size = 16

##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_Classifier(input_vec_shape,
                    dense_decode_units = 128, ## Dense layer parameters,
                    dense_layers = 2,
                    prob = 0.5, learn_rate = 0.0005, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
    beta = 0.001
    
    input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
    ######################################################################################################
    ########  CNN layers  ################################################################################
    ######################################################################################################
    
    cnn = tf.keras.layers.Conv1D(filters = 25, 
                                 kernel_size = 100,
                                 strides = 5,
                                 kernel_regularizer = tf.keras.regularizers.l2(beta)
                                )(input1)
#     cnn = tf.keras.layers.Activation('relu')(cnn)
#     cnn = tf.keras.layers.MaxPool1D(pool_size = 10, strides = 10)(cnn)
#     cnn = tf.keras.layers.Dropout(prob)(cnn)
    
    cnn = tf.keras.layers.Conv1D(filters = 25, 
                                 kernel_size = 50,
                                 strides = 5,
                                 kernel_regularizer = tf.keras.regularizers.l2(beta)
                                )(cnn)
#     cnn = tf.keras.layers.Activation('relu')(cnn)
#     cnn = tf.keras.layers.MaxPool1D(pool_size = 5, strides = 5)(cnn)
#     cnn = tf.keras.layers.Dropout(prob)(cnn)
    
    cnn = tf.keras.layers.Flatten()(cnn)
    
    cnn = tf.keras.layers.GaussianNoise(stddev=0.001)(cnn)
    
    ######################################################################################################
    ########  Classifier  ################################################################################
    ######################################################################################################
    
    y = tf.keras.layers.Dense(dense_decode_units, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta),
                              activation = 'relu'
                             )(cnn)
    y = tf.keras.layers.BatchNormalization()(y)
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.GaussianNoise(stddev=0.001)(y)
    
    y = tf.keras.layers.Dense(int(dense_decode_units/2), 
                              kernel_regularizer = tf.keras.regularizers.l2(beta),
                              activation = 'relu'
                             )(y)
    y = tf.keras.layers.BatchNormalization()(y)
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.GaussianNoise(stddev=0.001)(y)
    
    y = tf.keras.layers.Dense(1, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'sigmoid'
                             )(y)

    ## Generate Model from input and output
    model = tf.keras.models.Model(inputs=input1, outputs=y)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss)

    return model

In [10]:
DLNN_Classifier((2000, 1)).summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 381, 25)           2525      
                                                                 
 conv1d_1 (Conv1D)           (None, 67, 25)            31275     
                                                                 
 flatten (Flatten)           (None, 1675)              0         
                                                                 
 gaussian_noise (GaussianNoi  (None, 1675)             0         
 se)                                                             
                                                                 
 dense (Dense)               (None, 128)               214528    
                                                             

# Train data preparation

In [11]:
##################################################################################
##### Read CSV data
##################################################################################

for i in range(kgap_max+1):
    
    current_train_data_filepath = os.path.join(input_data_folder, train_data_filename.format(i))
    current_train_data = pd.read_csv(current_train_data_filepath, sep=',', header=0)
    current_train_data = current_train_data.drop('label', axis=1)
    
    if i == 0:
        train_data = current_train_data
    else:
        train_data = pd.merge(
            train_data,
            current_train_data,
            how="inner",
            on='nameseq'
        )

train_data['label'] = pd.Series([int(val.split('_')[-2])
                                 for val in train_data['nameseq']])

train_data = train_data.drop('nameseq', axis=1)

##################################################################################
##### Extract features and labels, create folds
##################################################################################

train_features = np.array(train_data.drop('label', axis=1))
train_features = train_features[:, :, np.newaxis]
train_labels = np.array(train_data['label'])
train_labels = train_labels.reshape((train_labels.shape[0], 1))

input_vec_shape = train_features[0].shape

folds = build_kfold(train_features, train_labels, k=n_fold, shuffle=shuffle, seed=seed)

## Write the k-fold dataset to file
foldPath = os.path.join(outPath, expName, "{}fold".format(n_fold))
if(not os.path.isdir(foldPath)):
    os.makedirs(foldPath)
pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

# Training

In [12]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Train/Test model on all folds, generate evaluations
##################################################################################

## Create and set directory to save model
modelPath = os.path.join(outPath, expName, "{}fold".format(n_fold), "models")
if(not os.path.isdir(modelPath)):
    os.makedirs(modelPath)

i = -1
for fold in folds:
    i += 1
    
    print("\nTrain/Test model on Fold #"+str(i)+".")
    
    model = DLNN_Classifier(input_vec_shape = input_vec_shape)
    
    ## Define the model callbacks for early stopping and saving the model. Then train model
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    modelCallbacks = [
        tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                           monitor = 'val_loss', verbose = 1, save_best_only = True, 
                                           save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
    ]
    
    # adding random shuffling of the dataset for training purpose
    index_arr = np.arange(fold["X_train"].shape[0])
    index_arr = np.random.permutation(index_arr)
    
    model.fit(x = fold["X_train"][index_arr], y = fold["y_train"][index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
              callbacks = modelCallbacks, validation_data = (fold["X_test"], fold["y_test"]))
    
    model = tf.keras.models.load_model(current_model_path)
    
    ##################################################################################
    ##### Prediction and metrics for TRAIN dataset
    ##################################################################################

    y_pred = model.predict(fold["X_train"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_train"], label_pred)
    prec = precision_score(fold["y_train"],label_pred)
    mcc = matthews_corrcoef(fold["y_train"], label_pred)

    conf = confusion_matrix(fold["y_train"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
    auc = roc_auc_score(fold["y_train"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Train")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    ##################################################################################
    ##### Prediction and metrics for TEST dataset
    ##################################################################################

    y_pred = model.predict(fold["X_test"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_test"], label_pred)
    prec = precision_score(fold["y_test"],label_pred)
    mcc = matthews_corrcoef(fold["y_test"], label_pred)

    conf = confusion_matrix(fold["y_test"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
    auc = roc_auc_score(fold["y_test"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Test")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    del model
    tf.keras.backend.clear_session()


Train/Test model on Fold #0.
Epoch 1/100
120/120 [==============================] - ETA: 0s - loss: 1.2382 - accuracy: 0.5879
Epoch 1: val_loss improved from inf to 0.96015, saving model to Results\NT_Site_PredNTS_Classification_DLNN_Kgap_CNN\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 4s 10ms/step - loss: 1.2382 - accuracy: 0.5879 - val_loss: 0.9601 - val_accuracy: 0.7610
Epoch 2/100
115/120 [===========================>..] - ETA: 0s - loss: 0.9825 - accuracy: 0.7027
Epoch 2: val_loss improved from 0.96015 to 0.87524, saving model to Results\NT_Site_PredNTS_Classification_DLNN_Kgap_CNN\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 1s 8ms/step - loss: 0.9806 - accuracy: 0.7018 - val_loss: 0.8752 - val_accuracy: 0.8008
Epoch 3/100
117/120 [============================>.] - ETA: 0s - loss: 0.8954 - accuracy: 0.7425
Epoch 3: val_loss improved from 0.87524 to 0.81632, saving model to Results\NT_Site_PredNTS_Classification_DLN

Epoch 29/100
119/120 [============================>.] - ETA: 0s - loss: 0.3952 - accuracy: 0.9569
Epoch 29: val_loss did not improve from 0.77522
120/120 [==============================] - 1s 8ms/step - loss: 0.3956 - accuracy: 0.9564 - val_loss: 0.8824 - val_accuracy: 0.8176
Epoch 30/100
113/120 [===========================>..] - ETA: 0s - loss: 0.4053 - accuracy: 0.9497
Epoch 30: val_loss did not improve from 0.77522
120/120 [==============================] - 1s 7ms/step - loss: 0.4122 - accuracy: 0.9465 - val_loss: 0.9535 - val_accuracy: 0.8050
Epoch 31/100
113/120 [===========================>..] - ETA: 0s - loss: 0.4010 - accuracy: 0.9452
Epoch 31: val_loss did not improve from 0.77522
120/120 [==============================] - 1s 7ms/step - loss: 0.3994 - accuracy: 0.9470 - val_loss: 0.9120 - val_accuracy: 0.8092
Epoch 32/100
120/120 [==============================] - ETA: 0s - loss: 0.3938 - accuracy: 0.9596
Epoch 32: val_loss did not improve from 0.77522
120/120 [==============

Epoch 59/100
120/120 [==============================] - ETA: 0s - loss: 0.3328 - accuracy: 0.9633
Epoch 59: val_loss did not improve from 0.77522
120/120 [==============================] - 1s 7ms/step - loss: 0.3328 - accuracy: 0.9633 - val_loss: 0.8957 - val_accuracy: 0.8176
Epoch 60/100
113/120 [===========================>..] - ETA: 0s - loss: 0.3495 - accuracy: 0.9552
Epoch 60: val_loss did not improve from 0.77522
120/120 [==============================] - 1s 7ms/step - loss: 0.3528 - accuracy: 0.9538 - val_loss: 0.8501 - val_accuracy: 0.8092
Epoch 61/100
120/120 [==============================] - ETA: 0s - loss: 0.3459 - accuracy: 0.9549
Epoch 61: val_loss did not improve from 0.77522
120/120 [==============================] - 1s 7ms/step - loss: 0.3459 - accuracy: 0.9549 - val_loss: 0.8533 - val_accuracy: 0.8050
Epoch 62/100
119/120 [============================>.] - ETA: 0s - loss: 0.3338 - accuracy: 0.9564
Epoch 62: val_loss did not improve from 0.77522
120/120 [==============

Epoch 89/100
118/120 [============================>.] - ETA: 0s - loss: 0.3009 - accuracy: 0.9682
Epoch 89: val_loss did not improve from 0.77522
120/120 [==============================] - 1s 8ms/step - loss: 0.3018 - accuracy: 0.9675 - val_loss: 0.9112 - val_accuracy: 0.8092
Epoch 90/100
120/120 [==============================] - ETA: 0s - loss: 0.3210 - accuracy: 0.9596
Epoch 90: val_loss did not improve from 0.77522
120/120 [==============================] - 1s 8ms/step - loss: 0.3210 - accuracy: 0.9596 - val_loss: 0.8578 - val_accuracy: 0.8134
Epoch 91/100
118/120 [============================>.] - ETA: 0s - loss: 0.3148 - accuracy: 0.9656
Epoch 91: val_loss did not improve from 0.77522
120/120 [==============================] - 1s 8ms/step - loss: 0.3149 - accuracy: 0.9654 - val_loss: 0.7943 - val_accuracy: 0.8134
Epoch 92/100
117/120 [============================>.] - ETA: 0s - loss: 0.3201 - accuracy: 0.9594
Epoch 92: val_loss did not improve from 0.77522
120/120 [==============

Epoch 17/100
120/120 [==============================] - ETA: 0s - loss: 0.4600 - accuracy: 0.9349
Epoch 17: val_loss did not improve from 0.81563
120/120 [==============================] - 1s 7ms/step - loss: 0.4600 - accuracy: 0.9349 - val_loss: 0.9976 - val_accuracy: 0.7715
Epoch 18/100
119/120 [============================>.] - ETA: 0s - loss: 0.4447 - accuracy: 0.9449
Epoch 18: val_loss did not improve from 0.81563
120/120 [==============================] - 1s 8ms/step - loss: 0.4450 - accuracy: 0.9449 - val_loss: 1.0009 - val_accuracy: 0.7904
Epoch 19/100
116/120 [============================>.] - ETA: 0s - loss: 0.4207 - accuracy: 0.9526
Epoch 19: val_loss did not improve from 0.81563
120/120 [==============================] - 1s 8ms/step - loss: 0.4198 - accuracy: 0.9528 - val_loss: 1.0194 - val_accuracy: 0.7925
Epoch 20/100
116/120 [============================>.] - ETA: 0s - loss: 0.4058 - accuracy: 0.9510
Epoch 20: val_loss did not improve from 0.81563
120/120 [==============

...

Epoch 82/100
113/120 [===========================>..] - ETA: 0s - loss: 0.3518 - accuracy: 0.9635
Epoch 82: val_loss did not improve from 0.88236
120/120 [==============================] - 1s 7ms/step - loss: 0.3522 - accuracy: 0.9627 - val_loss: 1.1445 - val_accuracy: 0.7626
Epoch 83/100
113/120 [===========================>..] - ETA: 0s - loss: 0.3374 - accuracy: 0.9624
Epoch 83: val_loss did not improve from 0.88236
120/120 [==============================] - 1s 7ms/step - loss: 0.3398 - accuracy: 0.9617 - val_loss: 1.1424 - val_accuracy: 0.7647
Epoch 84/100
113/120 [===========================>..] - ETA: 0s - loss: 0.4132 - accuracy: 0.9375
Epoch 84: val_loss did not improve from 0.88236
120/120 [==============================] - 1s 7ms/step - loss: 0.4122 - accuracy: 0.9370 - val_loss: 1.0859 - val_accuracy: 0.7710
Epoch 85/100
113/120 [===========================>..] - ETA: 0s - loss: 0.3622 - accuracy: 0.9596
Epoch 85: val_loss did not improve from 0.88236
120/120 [==============

Epoch 10/100
120/120 [==============================] - ETA: 0s - loss: 0.5975 - accuracy: 0.8940
Epoch 10: val_loss did not improve from 0.86696
120/120 [==============================] - 1s 7ms/step - loss: 0.5975 - accuracy: 0.8940 - val_loss: 0.8998 - val_accuracy: 0.7731
Epoch 11/100
119/120 [============================>.] - ETA: 0s - loss: 0.5809 - accuracy: 0.9028
Epoch 11: val_loss did not improve from 0.86696
120/120 [==============================] - 1s 7ms/step - loss: 0.5807 - accuracy: 0.9029 - val_loss: 0.9173 - val_accuracy: 0.7584
Epoch 12/100
119/120 [============================>.] - ETA: 0s - loss: 0.5521 - accuracy: 0.9133
Epoch 12: val_loss did not improve from 0.86696
120/120 [==============================] - 1s 7ms/step - loss: 0.5536 - accuracy: 0.9129 - val_loss: 0.9438 - val_accuracy: 0.7626
Epoch 13/100
118/120 [============================>.] - ETA: 0s - loss: 0.5787 - accuracy: 0.9073
Epoch 13: val_loss did not improve from 0.86696
120/120 [==============

Epoch 40/100
113/120 [===========================>..] - ETA: 0s - loss: 0.4062 - accuracy: 0.9607
Epoch 40: val_loss did not improve from 0.86696
120/120 [==============================] - 1s 7ms/step - loss: 0.4063 - accuracy: 0.9612 - val_loss: 1.0039 - val_accuracy: 0.7941
Epoch 41/100
119/120 [============================>.] - ETA: 0s - loss: 0.3937 - accuracy: 0.9664
Epoch 41: val_loss did not improve from 0.86696
120/120 [==============================] - 1s 7ms/step - loss: 0.3936 - accuracy: 0.9664 - val_loss: 1.0512 - val_accuracy: 0.7857
Epoch 42/100
113/120 [===========================>..] - ETA: 0s - loss: 0.3878 - accuracy: 0.9652
Epoch 42: val_loss did not improve from 0.86696
120/120 [==============================] - 1s 8ms/step - loss: 0.3895 - accuracy: 0.9633 - val_loss: 1.0705 - val_accuracy: 0.7773
Epoch 43/100
116/120 [============================>.] - ETA: 0s - loss: 0.4170 - accuracy: 0.9596
Epoch 43: val_loss did not improve from 0.86696
120/120 [==============

Epoch 70/100
118/120 [============================>.] - ETA: 0s - loss: 0.3521 - accuracy: 0.9672
Epoch 70: val_loss did not improve from 0.86696
120/120 [==============================] - 1s 8ms/step - loss: 0.3523 - accuracy: 0.9669 - val_loss: 0.9761 - val_accuracy: 0.7857
Epoch 71/100
119/120 [============================>.] - ETA: 0s - loss: 0.4036 - accuracy: 0.9543
Epoch 71: val_loss did not improve from 0.86696
120/120 [==============================] - 1s 7ms/step - loss: 0.4035 - accuracy: 0.9544 - val_loss: 1.0100 - val_accuracy: 0.7920
Epoch 72/100
120/120 [==============================] - ETA: 0s - loss: 0.4008 - accuracy: 0.9533
Epoch 72: val_loss did not improve from 0.86696
120/120 [==============================] - 1s 7ms/step - loss: 0.4008 - accuracy: 0.9533 - val_loss: 0.9913 - val_accuracy: 0.7920
Epoch 73/100
113/120 [===========================>..] - ETA: 0s - loss: 0.3662 - accuracy: 0.9646
Epoch 73: val_loss did not improve from 0.86696
120/120 [==============

Epoch 100/100
116/120 [============================>.] - ETA: 0s - loss: 0.3468 - accuracy: 0.9650
Epoch 100: val_loss did not improve from 0.86696
120/120 [==============================] - 1s 8ms/step - loss: 0.3485 - accuracy: 0.9638 - val_loss: 0.9694 - val_accuracy: 0.8025


## k-fold Training evaluation

In [13]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Test,0.790061,0.784380,0.857217,0.806002,0.774066,0.582841
Train,0.932308,0.923794,0.982727,0.943328,0.921292,0.865506


In [14]:
evaluations_df[evaluations_df["Train_Test"] == 'Test']

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
1,0,Test,0.844864,0.836735,"[0.0, 0.0041841004184100415, 0.096234309623430...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.008403361344537815...","[1.9991492, 0.9991492, 0.9913033, 0.9911901, 0...",0.895380,0.857741,0.831933,0.689928
3,1,Test,0.809224,0.845070,"[0.0, 0.004201680672268907, 0.0714285714285714...","[0.0, 0.0, 0.0, 0.0041841004184100415, 0.00418...","[1.9996018, 0.9996019, 0.99457043, 0.99252105,...",0.873651,0.756303,0.861925,0.621790
5,2,Test,0.771008,0.732852,"[0.0, 0.004201680672268907, 0.1008403361344537...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.999722, 0.99972194, 0.9904845, 0.9889319, 0...",0.849446,0.852941,0.689076,0.549444
7,3,Test,0.758403,0.763948,"[0.0, 0.004201680672268907, 0.0210084033613445...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9975613, 0.99756134, 0.99500966, 0.99050516...",0.821376,0.747899,0.768908,0.516921
9,4,Test,0.766807,0.743295,"[0.0, 0.004201680672268907, 0.0336134453781512...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9999307, 0.99993074, 0.99918777, 0.9990115,...",0.846233,0.815126,0.718487,0.536123


In [15]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Train,0.954856,0.951983,"[0.0, 0.0010504201680672268, 0.095588235294117...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9998939, 0.9998939, 0.99201, 0.9919518, 0.9...",0.992006,0.957983,0.951731,0.909730
1,0,Test,0.844864,0.836735,"[0.0, 0.0041841004184100415, 0.096234309623430...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.008403361344537815...","[1.9991492, 0.9991492, 0.9913033, 0.9911901, 0...",0.895380,0.857741,0.831933,0.689928
2,1,Train,0.940157,0.954496,"[0.0, 0.001049317943336831, 0.0398740818467995...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9999448, 0.9999448, 0.9972224, 0.9971629, 0...",0.988341,0.924449,0.955882,0.880754
3,1,Test,0.809224,0.845070,"[0.0, 0.004201680672268907, 0.0714285714285714...","[0.0, 0.0, 0.0, 0.0041841004184100415, 0.00418...","[1.9996018, 0.9996019, 0.99457043, 0.99252105,...",0.873651,0.756303,0.861925,0.621790
4,2,Train,0.925498,0.899507,"[0.0, 0.001049317943336831, 0.1101783840503672...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9997125, 0.9997124, 0.9866214, 0.98631454, ...",0.980735,0.958027,0.892970,0.852804
5,2,Test,0.771008,0.732852,"[0.0, 0.004201680672268907, 0.1008403361344537...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.999722, 0.99972194, 0.9904845, 0.9889319, 0...",0.849446,0.852941,0.689076,0.549444
6,3,Train,0.913431,0.913866,"[0.0, 0.001049317943336831, 0.0545645330535152...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00104931...","[1.9984024, 0.9984023, 0.9899941, 0.9897899, 0...",0.968823,0.912907,0.913956,0.826863
7,3,Test,0.758403,0.763948,"[0.0, 0.004201680672268907, 0.0210084033613445...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9975613, 0.99756134, 0.99500966, 0.99050516...",0.821376,0.747899,0.768908,0.516921
8,4,Train,0.927597,0.899119,"[0.0, 0.001049317943336831, 0.3126967471143757...","[0.0, 0.0, 0.0, 0.001049317943336831, 0.001049...","[1.9999937, 0.9999937, 0.98466945, 0.98461896,...",0.983730,0.963274,0.891920,0.857380
9,4,Test,0.766807,0.743295,"[0.0, 0.004201680672268907, 0.0336134453781512...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9999307, 0.99993074, 0.99918777, 0.9990115,...",0.846233,0.815126,0.718487,0.536123


# Independent data

In [16]:
##################################################################################
##### Read CSV data
##################################################################################

for i in range(kgap_max+1):

    current_indpe_data_filepath = os.path.join(input_data_folder, indpe_data_filename.format(i))
    current_indpe_data = pd.read_csv(current_indpe_data_filepath, sep=',', header=0)
    current_indpe_data = current_indpe_data.drop('label', axis=1)
    
    if i == 0:
        indpe_data = current_indpe_data
    else:
        indpe_data = pd.merge(
            indpe_data,
            current_indpe_data,
            how="inner",
            on='nameseq'
        )

indpe_data['label'] = pd.Series([int(val.split('_')[-2])
                                 for val in indpe_data['nameseq']])

indpe_data = indpe_data.drop('nameseq', axis=1)

##################################################################################
##### Extract features and labels, create folds
##################################################################################

indpe_features = np.array(indpe_data.drop('label', axis=1))
indpe_features = indpe_features[:, :, np.newaxis]
indpe_labels = np.array(indpe_data['label'])
indpe_labels = indpe_labels.reshape((indpe_labels.shape[0], 1))

## Using k-fold Models

### Performance of each k-fold model

In [17]:
## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    label_pred = pred2label(y_pred)

    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(indpe_labels, label_pred)
    prec = precision_score(indpe_labels,label_pred)
    mcc = matthews_corrcoef(indpe_labels, label_pred)

    conf = confusion_matrix(indpe_labels, label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
    auc = roc_auc_score(indpe_labels, y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Independent")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    del model
    tf.keras.backend.clear_session()

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.605878,0.24089,0.658649,0.634483,0.600196,0.176786


In [18]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Independent,0.613878,0.243346,"[0.0, 0.0049261083743842365, 0.009852216748768...","[0.0, 0.0, 0.0, 0.0009784735812133072, 0.00097...","[1.9995866, 0.99958664, 0.99814665, 0.9980044,...",0.669893,0.630542,0.610568,0.181116
1,1,Independent,0.661224,0.266520,"[0.0, 0.0, 0.0, 0.009852216748768473, 0.009852...","[0.0, 0.0009784735812133072, 0.001956947162426...","[1.9997094, 0.9997094, 0.99965346, 0.99956363,...",0.665401,0.596059,0.674168,0.208040
2,2,Independent,0.565714,0.227197,"[0.0, 0.0049261083743842365, 0.004926108374384...","[0.0, 0.0, 0.0009784735812133072, 0.0009784735...","[1.9997826, 0.9997826, 0.9979327, 0.9975898, 0...",0.657814,0.674877,0.544031,0.162810
3,3,Independent,0.614694,0.233663,"[0.0, 0.0, 0.0, 0.0049261083743842365, 0.00492...","[0.0, 0.0009784735812133072, 0.003913894324853...","[1.9992101, 0.9992101, 0.99740183, 0.99709284,...",0.635964,0.581281,0.621331,0.153047
4,4,Independent,0.573878,0.233723,"[0.0, 0.0049261083743842365, 0.009852216748768...","[0.0, 0.0, 0.0, 0.004892367906066536, 0.004892...","[1.999966, 0.999966, 0.9999298, 0.99949825, 0....",0.664171,0.689655,0.550881,0.178917


### Mean score with k-fold models

In [19]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    total_pred += y_pred
    all_preds.append(y_pred)
    
    del model
    tf.keras.backend.clear_session()
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.621224,0.252372,0.678352,0.655172,0.614481,0.20251


### Voting score with k-fold models

In [20]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    vote_pred = pred2label(y_pred)
    total_pred += vote_pred
    all_preds.append(vote_pred)
    
    del model
    tf.keras.backend.clear_session()
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.622041,0.25,0.665972,0.640394,0.618395,0.194681


## Using New Model

Train model on full data from training. Predict and evaluate on Independent data.

In [21]:
model = DLNN_Classifier(input_vec_shape = input_vec_shape)
    
## Define the model callbacks for early stopping and saving the model. Then train model
current_model_path = os.path.join(modelPath, "_fullModel.hdf5")
modelCallbacks = [
    tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                       monitor = 'val_loss', verbose = 1, save_best_only = True, 
                                       save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
]

# adding random shuffling of the dataset for training purpose
index_arr = np.arange(train_features.shape[0])
index_arr = np.random.permutation(index_arr)

model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
          callbacks = modelCallbacks, validation_data = (indpe_features, indpe_labels))
# model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
#           callbacks = modelCallbacks, validation_split = 0.2)

model = tf.keras.models.load_model(current_model_path)

Epoch 1/100
149/149 [==============================] - ETA: 0s - loss: 1.1710 - accuracy: 0.6146
Epoch 1: val_loss improved from inf to 1.21000, saving model to Results\NT_Site_PredNTS_Classification_DLNN_Kgap_CNN\5fold\models\_fullModel.hdf5
149/149 [==============================] - 2s 11ms/step - loss: 1.1710 - accuracy: 0.6146 - val_loss: 1.2100 - val_accuracy: 0.2841
Epoch 2/100
145/149 [============================>.] - ETA: 0s - loss: 0.9534 - accuracy: 0.7224
Epoch 2: val_loss improved from 1.21000 to 1.18595, saving model to Results\NT_Site_PredNTS_Classification_DLNN_Kgap_CNN\5fold\models\_fullModel.hdf5
149/149 [==============================] - 1s 8ms/step - loss: 0.9547 - accuracy: 0.7229 - val_loss: 1.1860 - val_accuracy: 0.5224
Epoch 3/100
149/149 [==============================] - ETA: 0s - loss: 0.8728 - accuracy: 0.7678
Epoch 3: val_loss improved from 1.18595 to 1.14827, saving model to Results\NT_Site_PredNTS_Classification_DLNN_Kgap_CNN\5fold\models\_fullModel.hdf5


Epoch 30/100
144/149 [===========================>..] - ETA: 0s - loss: 0.4060 - accuracy: 0.9475
Epoch 30: val_loss did not improve from 1.14827
149/149 [==============================] - 1s 8ms/step - loss: 0.4070 - accuracy: 0.9458 - val_loss: 1.6301 - val_accuracy: 0.6392
Epoch 31/100
144/149 [===========================>..] - ETA: 0s - loss: 0.4178 - accuracy: 0.9423
Epoch 31: val_loss did not improve from 1.14827
149/149 [==============================] - 1s 8ms/step - loss: 0.4165 - accuracy: 0.9421 - val_loss: 1.8090 - val_accuracy: 0.6188
Epoch 32/100
148/149 [============================>.] - ETA: 0s - loss: 0.3960 - accuracy: 0.9497
Epoch 32: val_loss did not improve from 1.14827
149/149 [==============================] - 1s 8ms/step - loss: 0.3961 - accuracy: 0.9492 - val_loss: 1.7903 - val_accuracy: 0.6359
Epoch 33/100
143/149 [===========================>..] - ETA: 0s - loss: 0.4079 - accuracy: 0.9449
Epoch 33: val_loss did not improve from 1.14827
149/149 [==============

Epoch 60/100
142/149 [===========================>..] - ETA: 0s - loss: 0.3485 - accuracy: 0.9533
Epoch 60: val_loss did not improve from 1.14827
149/149 [==============================] - 1s 8ms/step - loss: 0.3470 - accuracy: 0.9542 - val_loss: 1.6053 - val_accuracy: 0.6522
Epoch 61/100
148/149 [============================>.] - ETA: 0s - loss: 0.3675 - accuracy: 0.9527
Epoch 61: val_loss did not improve from 1.14827
149/149 [==============================] - 1s 8ms/step - loss: 0.3681 - accuracy: 0.9521 - val_loss: 1.7256 - val_accuracy: 0.6286
Epoch 62/100
144/149 [===========================>..] - ETA: 0s - loss: 0.3537 - accuracy: 0.9553
Epoch 62: val_loss did not improve from 1.14827
149/149 [==============================] - 1s 8ms/step - loss: 0.3548 - accuracy: 0.9547 - val_loss: 1.5553 - val_accuracy: 0.6384
Epoch 63/100
143/149 [===========================>..] - ETA: 0s - loss: 0.3697 - accuracy: 0.9480
Epoch 63: val_loss did not improve from 1.14827
149/149 [==============

Epoch 90/100
149/149 [==============================] - ETA: 0s - loss: 0.3311 - accuracy: 0.9589
Epoch 90: val_loss did not improve from 1.14827
149/149 [==============================] - 1s 8ms/step - loss: 0.3311 - accuracy: 0.9589 - val_loss: 1.8809 - val_accuracy: 0.6106
Epoch 91/100
144/149 [===========================>..] - ETA: 0s - loss: 0.3403 - accuracy: 0.9605
Epoch 91: val_loss did not improve from 1.14827
149/149 [==============================] - 1s 8ms/step - loss: 0.3381 - accuracy: 0.9618 - val_loss: 1.6086 - val_accuracy: 0.6563
Epoch 92/100
144/149 [===========================>..] - ETA: 0s - loss: 0.3163 - accuracy: 0.9648
Epoch 92: val_loss did not improve from 1.14827
149/149 [==============================] - 1s 8ms/step - loss: 0.3189 - accuracy: 0.9635 - val_loss: 1.8547 - val_accuracy: 0.5984
Epoch 93/100
144/149 [===========================>..] - ETA: 0s - loss: 0.3374 - accuracy: 0.9579
Epoch 93: val_loss did not improve from 1.14827
149/149 [==============

In [22]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

y_pred = model.predict(indpe_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
auc = roc_auc_score(indpe_labels, y_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.601633,0.233645,0.650174,0.615764,0.598826,0.160872


In [23]:
print(classification_report(indpe_labels, np.round(y_pred).astype(int)))

              precision    recall  f1-score   support

           0       0.89      0.60      0.71      1022
           1       0.23      0.62      0.34       203

    accuracy                           0.60      1225
   macro avg       0.56      0.61      0.53      1225
weighted avg       0.78      0.60      0.65      1225

